In [ ]:
%pip install transformers

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel

MODEL_NAME = 'huawei-noah/TinyBERT_General_4L_312D'

model = TFAutoModel.from_pretrained(MODEL_NAME, from_pt = True)

In [5]:
callable = tf.function(model.call)
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, 384], tf.int32, name="input_ids"),tf.TensorSpec([None, 384], tf.int32, name="attention_mask")])

tf.saved_model.save(model, 'tiny2_am', signatures=concrete_function)
!saved_model_cli show --dir tiny2_am --tag_set serve --signature_def serving_default

2022-02-18 10:50:22.494238: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: tiny2_am/assets


INFO:tensorflow:Assets written to: tiny2_am/assets


The given SavedModel SignatureDef contains the following input(s):
  inputs['attention_mask'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_attention_mask:0
  inputs['input_ids'] tensor_info:
      dtype: DT_INT32
      shape: (-1, 384)
      name: serving_default_input_ids:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['last_hidden_state'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 384, 312)
      name: StatefulPartitionedCall:0
  outputs['pooler_output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 312)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_format=tfjs_graph_model \
    --signature_name=serving_default \
    --saved_model_tags=serve \
    /Users/danielgoldelman/Desktop/privacy-tech-lab/modelrepo/tiny2/tiny2_am \
    /Users/danielgoldelman/Desktop/privacy-tech-lab/modelrepo/tiny2/tiny2_am_web